In [ ]:
%pip install google-cloud-speech

In [1]:
import pyaudio

p = pyaudio.PyAudio()

print("Available audio devices:")
for i in range(p.get_device_count()):
    device_info = p.get_device_info_by_index(i)
    print(f"Device {i}: {device_info['name']}")

p.terminate()


Available audio devices:
Device 0: Rohan’s Iphone Microphone
Device 1: Built-in Microphone
Device 2: Built-in Output


In [3]:
%pip install sounddevice


Note: you may need to restart the kernel to use updated packages.


In [7]:
import sounddevice as sd

device_index = 1  # Replace with the correct index for Built-in Microphone
sd.default.device = (device_index, None)

print("Recording for 5 seconds...")
data = sd.rec(int(5 * 44100), samplerate=44100, channels=1, dtype='int16')
sd.wait()
print("Recording complete.")


Recording for 5 seconds...
||PaMacCore (AUHAL)|| Error on line 1277: err='-66748', msg=Unknown Error


PortAudioError: Error opening InputStream: Internal PortAudio error [PaErrorCode -9986]

In [5]:
import sounddevice as sd

print(sd.query_devices())


  0 Rohan’s Iphone Microphone, Core Audio (1 in, 0 out)
> 1 Built-in Microphone, Core Audio (2 in, 0 out)
< 2 Built-in Output, Core Audio (0 in, 2 out)


In [2]:
import pyaudio

p = pyaudio.PyAudio()


print(p.get_device_info_by_index(1))


try:
    stream = p.open(
        format=pyaudio.paInt16,
        channels=1,
        rate=44100,  # Change if your microphone supports a different rate
        input=True,
        input_device_index=1,  # Use the index for Built-in Microphone
        frames_per_buffer=1024
    )
    print("Using Built-in Microphone...")
finally:
    stream.stop_stream()
    stream.close()
    p.terminate()


{'index': 1, 'structVersion': 2, 'name': 'Built-in Microphone', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.0029478458049886623, 'defaultLowOutputLatency': 0.01, 'defaultHighInputLatency': 0.01310657596371882, 'defaultHighOutputLatency': 0.1, 'defaultSampleRate': 44100.0}
||PaMacCore (AUHAL)|| Error on line 1277: err='-66748', msg=Unknown Error


NameError: name 'stream' is not defined

In [5]:
import os
import pyaudio
from google.cloud import speech
from six.moves import queue

# Set your Google Cloud credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/rgopalam/Desktop/seismic-rarity-427422-p7-ab3b4a8726ef.json"  # Replace with your JSON key file path

# Audio recording parameters
RATE = 16000
CHUNK = int(RATE / 10)  # 100ms

class MicrophoneStream:
    """Opens a recording stream as a generator yielding audio chunks."""
    def __init__(self, rate, chunk):
        self._rate = rate
        self._chunk = chunk
        self._buff = queue.Queue()
        self.closed = True

    def __enter__(self):
        self._audio_interface = pyaudio.PyAudio()
        self._audio_stream = self._audio_interface.open(
            format=pyaudio.paInt16,
            channels=1,
            rate=self._rate,
            input=True,
            frames_per_buffer=self._chunk,
            stream_callback=self._fill_buffer,
        )
        self.closed = False
        return self

    def __exit__(self, type, value, traceback):
        self._audio_stream.stop_stream()
        self._audio_stream.close()
        self.closed = True
        self._buff.put(None)
        self._audio_interface.terminate()

    def _fill_buffer(self, in_data, frame_count, time_info, status_flags):
        """Continuously collect data from the audio stream."""
        self._buff.put(in_data)
        return None, pyaudio.paContinue

    def generator(self):
        while not self.closed:
            chunk = self._buff.get()
            if chunk is None:
                return
            yield chunk

def transcribe_stream():
    """Streams audio to Google Cloud Speech-to-Text and prints the transcriptions."""
    client = speech.SpeechClient()

    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=RATE,
        language_code="en-US",
    )
    streaming_config = speech.StreamingRecognitionConfig(config=config, interim_results=True)

    with MicrophoneStream(RATE, CHUNK) as stream:
        audio_generator = stream.generator()
        requests = (speech.StreamingRecognizeRequest(audio_content=content) for content in audio_generator)

        responses = client.streaming_recognize(streaming_config, requests)

        print("Listening for audio... Press Ctrl+C to stop.")
        try:
            for response in responses:
                if not response.results:
                    continue

                result = response.results[0]
                if result.is_final:
                    print(f"Final: {result.alternatives[0].transcript}")
                else:
                    print(f"Interim: {result.alternatives[0].transcript}")
        except KeyboardInterrupt:
            print("Stopped by user.")

if __name__ == "__main__":
    transcribe_stream()


||PaMacCore (AUHAL)|| Error on line 1277: err='-66748', msg=Unknown Error


OSError: [Errno -9986] Internal PortAudio error